# Lecture 21
## Tuesday, November 13th 2018
### Joins with `SQLite`, `pandas`

## Starting Up
You can connect to the saved database from last time if you want.  Alternatively, for extra practice, you can just recreate it from the datasets provided in the `.txt` files.  That's what I'll do.

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L21DB_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

## Recap
Last time, you played with a bunch of `SQLite` commands to query and update the tables in the database.

One thing we didn't get to was how to query the contributors table based off of a query in the candidates table.  For example, suppose you want to query which contributors donated to Obama. You could use a nested `SELECT` statement to accomplish that.

In [2]:
query = '''SELECT * FROM contributors WHERE candidate_id = (SELECT id from candidates WHERE last_name = "Obama")'''
viz_tables(contributor_cols, query)

,amount,candidate_id,city,date,first_name,id,last_name,middle_name,state,street_1,street_2,zip
0,50.00,20,Dana Point,2007-07-30,Steve,26,Buckler,,CA,24351 Armada Dr,,926291306
1,25.00,20,Dana Point,2007-08-16,Steve,27,Buckler,,CA,24351 Armada Dr,,926291306
2,100.00,20,FAIRFAX,2007-09-19,Bruce,28,Buckheit,,VA,8904 KAREN DR,,220312731
3,2300.00,20,Park City,2007-08-14,Linda,29,Buckel,,UT,PO Box 683130,,840683130
4,-2300.00,20,Park City,2007-08-14,Linda,30,Buckel,,UT,PO Box 683130,,840683130
5,4600.00,20,Park City,2007-08-14,Linda,31,Buckel,,UT,PO Box 683130,,840683130
6,100.00,20,Kansas City,2007-09-25,Thomas,32,Buck,,MO,4206 Terrace Street,,64111
7,200.00,20,Northfield,2007-09-12,Jay,33,Buck,K.,IL,1855 Old Willow Rd Unit 322,,600932918
8,2300.00,20,Camden,2007-09-30,Blaine,34,Buck,M,ME,45 Eaton Ave,,48431752
9,50.00,20,North Miami,2007-09-13,Barbara,35,Buck,,FL,1780 NE 138th St,,331811316


# Joins
The last example involved querying data from multiple tables.

In particular, we combined columns from the two related tables (related through the `FOREIGN KEY`).

This leads to the idea of *joining* multiple tables together.  `SQL` has a set of commands to handle different types of joins.  `SQLite` does not support the full suite of join commands offered by `SQL` but you should still be able to get the main ideas from the limited command set.

We'll begin with the `INNER JOIN`.

## INNER JOIN
The idea here is that you will combine the tables if the values of certain columns are the same between the two tables.  In our example, we will join the two tables based on the candidate id.  The result of the `INNER JOIN` will be a new table consisting of the columns we requested and containing the common data.  Since we are joining based off of the candidate id, we will not be excluding any rows.

### Example

Here are two tables.  Table A has the form:

| nA   | attr  | idA |
| :::: | ::::: | ::: |
| s1   | 23    | 0   |
| s2   | 7     | 2   |

and table B has the form:

| nB   | attr  | idB |
| :::: | ::::: | ::: |
| t1   | 60    | 0   |
| t2   | 14    | 7   |
| t3   | 22    | 2   |

Table A is associated with Table B through a foreign key on the id column.

If we join the two tables by comparing the id columns and selecting the nA, nB, and attr columns then we'll get 

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| s2   | 7       | t3  | 22     |


The `SQLite` code to do this join would be 

```SQL
SELECT nA, A.attr, nB, B.attr FROM A INNER JOIN B ON B.idB = A.idA
```

Notice that the second row in table B is gone because the id values are not the same.

### Thoughts
What is `SQL` doing with this operation?  It may help to visualize this with a Venn diagram.  Table A has rows with values corresponding to the `idA` attribute.  Column B has rows with values corresponding to the `idB` attribute.  The `INNER JOIN` will combine the two tables such that rows with common entries in the `id` attributes are included.  We essentially have the following Venn diagram.

![](figs/inner_join.png)

## `LEFT JOIN` or `LEFT OUTER JOIN`
There are many ways to combine two tables.  We just explored one possibility in which we combined the tables based upon the intersection of the two tables (the `INNER JOIN`).

Now we'll talk about the `LEFT JOIN` or `LEFT OUTER JOIN`.

In words, the `LEFT JOIN` is combining the tables based upon what is in the intersection of the two tables *and* what is in the "reference" table.

We can consider our toy example in two guises:

#### Example A
Let's do a `LEFT JOIN` of table B from table A.  That is, we'd like to make a new table by putting table B into table A.  In this case, we'll consider table A our "reference" table.  We're comparing by the `id` column again.  We know that these two tables share ids 0 and 2 and table A doesn't have anything else in it.  The resulting table is:

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| s2   | 7       | t3  | 22     |

That's not very exciting.  It's the same result as from the `INNER JOIN`.  We can do another example that may be more enlightening.

#### Example B
Let's do a `LEFT JOIN` of table A from table B.  That is, we'd like to make a new table by putting table A into table B.  In this case, we'll consider table B our "reference" table.  Again, we use the `id` column from comparison.  We know that these two tables share ids 0 and 2.  This time, table B also contains the id 7, which is not shared by table A.  The resulting table is:

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| None | NaN     | t2  | 14     |
| s2   | 7       | t3  | 22     |

Notice that `SQLite` filed in the missing entries for us.  This is necessary for completion of the requested join.

The `SQLite` commands to accomplish all of this are:

```SQL
SELECT nA, A.attr, nB, B.attr FROM A LEFT JOIN B ON B.idB = A.idA
```

and

```SQL
SELECT nA, A.attr, nB, B.attr FROM B LEFT JOIN A ON A.idA = B.idB
```

Here is a visualization using Venn diagrams of the `LEFT JOIN`.

![](figs/left_join.png)

---

# `pandas`

We've been working with databases for the last few lectures and learning `SQLite` commands to work with and manipulate the databases.  There is a `Python` package called `pandas` that provides broad support for data structures.  It can be used to interact with relationsional databases through its own methods and even through `SQL` commands.

In the last part of this lecture, you will get to redo a bunch of the database exercises using `pandas`.

We won't be able to cover `pandas` from the ground up, but it's a well-documented library and is fairly easy to get up and running.  Here's the website:  [`pandas`](http://pandas.pydata.org/).

## Reading a datafile into `pandas`

In [3]:
# Using pandas naming convention
dfcand = pd.read_csv("candidates.txt", sep="|")
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


In [4]:
dfcontr = pd.read_csv("contributors.txt", sep="|")
dfcontr

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,NaN,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
6,NaN,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
7,NaN,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16
8,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16
9,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16


Reading things in is quite easy with `pandas`.

Notice that `pandas` populates empty fields with `NaN` values.

The `id` column in the contributors dataset is superfluous.  Let's delete it.

In [5]:
del dfcontr['id']
dfcontr.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


Very nice!  And we used the `head` method to print out the first five rows.

## Creating a Table with `pandas`
We can use `pandas` to create tables in a database.

First, let's create a new database since we've already done a lot on our test database.

In [6]:
dbp = sqlite3.connect('L21_pandas_DB.sqlite')
csr = dbp.cursor()
csr.execute("DROP TABLE IF EXISTS candidates")
csr.execute("DROP TABLE IF EXISTS contributors")
csr.execute("PRAGMA foreign_keys=1")

csr.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_name TEXT, 
               party TEXT NOT NULL)''')

dbp.commit() # Commit changes to the database

csr.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

dbp.commit()

Last time, we opened the data files with `Python` and then manually used `SQLite` commands to populate the individual tables.  We can use `pandas` instead like so.

In [7]:
dfcand.to_sql("candidates", dbp, if_exists="append", index=False)

How big is our table?

In [8]:
dfcand.shape

(17, 5)

We can visualize the data in our `pandas`-populated table.  No surprises here except that `pandas` did everything for us.

In [9]:
query = '''SELECT * FROM candidates'''
csr.execute(query).fetchall()

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

## Querying a table with `pandas`

### One Way

In [10]:
dfcand.query("first_name=='Mike' & party=='D'")

,id,first_name,last_name,middle_name,party
6,24,Mike,Gravel,NaN,D


### Another Way

In [11]:
dfcand[(dfcand.first_name=="Mike") & (dfcand.party=="D")]

,id,first_name,last_name,middle_name,party
6,24,Mike,Gravel,NaN,D


### More Queries

In [12]:
dfcand[dfcand.middle_name.notnull()]

,id,first_name,last_name,middle_name,party
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
16,41,Fred,Thompson,D.,R


In [13]:
dfcand[dfcand.first_name.isin(['Mike', 'Hillary'])]

,id,first_name,last_name,middle_name,party
2,34,Hillary,Clinton,R.,D
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R


## Sorting

In [25]:
dfcand.sort_values(by='party')

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
13,29,Bill,Richardson,NaN,D
6,24,Mike,Gravel,NaN,D
11,20,Barack,Obama,NaN,D
9,31,Dennis,Kucinich,NaN,D
14,35,Mitt,Romney,NaN,R
12,32,Ron,Paul,NaN,R


In [26]:
dfcand.sort_values(by='party', ascending=False)

,id,first_name,last_name,middle_name,party
8,30,Duncan,Hunter,NaN,R
7,16,Mike,Huckabee,NaN,R
15,38,Tom,Tancredo,NaN,R
14,35,Mitt,Romney,NaN,R
12,32,Ron,Paul,NaN,R
10,37,John,McCain,NaN,R
1,36,Samuel,Brownback,NaN,R
16,41,Fred,Thompson,D.,R
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D


## Selecting Columns

In [27]:
dfcand[['last_name', 'party']]

,last_name,party
0,Biden,D
1,Brownback,R
2,Clinton,D
3,Dodd,D
4,Edwards,D
5,Giuliani,R
6,Gravel,D
7,Huckabee,R
8,Hunter,R
9,Kucinich,D


In [28]:
dfcand[['last_name', 'party']].count()

last_name    17
party        17
dtype: int64

In [29]:
dfcand[['first_name']].drop_duplicates()

,first_name
0,Joseph
1,Samuel
2,Hillary
3,Christopher
4,John
5,Rudolph
6,Mike
8,Duncan
9,Dennis
11,Barack


In [30]:
dfcand[['first_name']].drop_duplicates().count()

first_name    15
dtype: int64

## Altering Tables

Creating a new column is quite easy with `pandas`.

In [36]:
dfcand['name'] = dfcand['last_name'] + ", " + dfcand['first_name']
dfcand

,id,first_name,last_name,middle_name,party,name
0,33,Joseph,Biden,NaN,D,"Biden, Joseph"
1,36,Samuel,Brownback,NaN,R,"Brownback, Samuel"
2,34,Hillary,Clinton,R.,D,"Clinton, Hillary"
3,39,Christopher,Dodd,J.,D,"Dodd, Christopher"
4,26,John,Edwards,NaN,D,"Edwards, John"
5,22,Rudolph,Giuliani,NaN,R,"Giuliani, Rudolph"
6,24,Mike,Gravel,NaN,D,"Gravel, Mike"
7,16,Mike,Huckabee,NaN,R,"Huckabee, Mike"
8,30,Duncan,Hunter,NaN,R,"Hunter, Duncan"
9,31,Dennis,Kucinich,NaN,D,"Kucinich, Dennis"


We can change an existing field as well.

In [37]:
dfcand.loc[dfcand.first_name == "Mike", "name"]

6      Gravel, Mike
7    Huckabee, Mike
Name: name, dtype: object

In [38]:
dfcand.loc[dfcand.first_name == "Mike", "name"] = "Mikey"

In [39]:
dfcand.query("first_name == 'Mike'")

,id,first_name,last_name,middle_name,party,name
6,24,Mike,Gravel,NaN,D,Mikey
7,16,Mike,Huckabee,NaN,R,Mikey


You may recall that `SQLite` doesn't have the functionality to drop a column.  It's a one-liner with `pandas`.

In [40]:
del dfcand['name']
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


## Aggregation
We'd like to get information about the tables such as the maximum amount contributed to the candidates.  Here are a bunch of ways to describe the tables.

In [45]:
dfcand.describe()

,id
count,17.000000
mean,30.764706
std,7.110990
min,16.000000
25%,26.000000
50%,32.000000
75%,36.000000
max,41.000000


It's not very interesting with the candidates table because the candidates table only has one numeric column.

I'll use the contributors table to do some demos now.

In [47]:
dfcontr.amount.max()

4600.0

In [48]:
dfcontr[dfcontr.amount==dfcontr.amount.max()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20


In [49]:
dfcontr.groupby("state").sum()

,zip,amount,candidate_id
state,,,
AK,2985459621,1210.00,111
AR,864790,14200.00,192
AZ,860011121,120.00,37
CA,14736360720,-5013.73,600
CO,2405477834,-5823.00,111
CT,68901376,2300.00,35
DC,800341853,-1549.91,102
FL,8970626520,-4050.00,803
IA,50266,250.00,16


In [50]:
dfcontr.groupby("state")["amount"].sum()

state
AK     1210.00
AR    14200.00
AZ      120.00
CA    -5013.73
CO    -5823.00
CT     2300.00
DC    -1549.91
FL    -4050.00
IA      250.00
ID     -261.00
IL    -5586.80
KS     -330.00
KY     -200.00
LA     1300.00
MA      -83.00
MD      300.00
ME     2520.00
MI    -1265.00
MN      322.00
MO      100.00
NC      500.00
NH      -24.60
NJ     -817.45
NV      725.00
NY    -6474.50
OH      450.00
OK      800.00
PA    -2146.00
RI      200.00
SC     2400.00
TN      -25.00
TX     1985.24
UT     5050.00
VA      515.92
WA     -500.00
Name: amount, dtype: float64

In [51]:
dfcontr.state.unique()

array(['VA', 'CA', 'AR', 'DC', 'SC', 'IA', 'OH', 'NC', 'UT', 'MO', 'IL',
       'ME', 'FL', 'MD', 'MI', 'CO', 'WA', 'NY', 'TX', 'KY', 'PA', 'TN',
       'MA', 'MN', 'KS', 'NJ', 'NH', 'ID', 'OK', nan, 'NV', 'CT', 'RI',
       'AK', 'LA', 'AZ'], dtype=object)

There is also a version of the `LIMIT` clause.  It's very intuitive with `pandas`.

In [52]:
dfcand[0:3]

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D


The usual `Python` slicing works just fine!

## Joins with `pandas`

`pandas` has some some documentation on `joins`:  [Merge, join, and concatenate](http://pandas.pydata.org/pandas-docs/stable/merging.html).  If you want some more reinforcement on the concepts from earlier regarding `JOIN`, then the `pandas` documentation may be a good place to get it.

You may also be interested in [a comparison with `SQL`](http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html#compare-with-sql-join).

To do joins with `pandas`, we use the `merge` command.

Here's an example of an explicit inner join:

In [53]:
cols_wanted = ['last_name_x', 'first_name_x', 'candidate_id', 'id', 'last_name_y']
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16,16,Huckabee
1,Ahrens,Don,16,16,Huckabee
2,Ahrens,Don,16,16,Huckabee
3,Ahrens,Don,16,16,Huckabee
4,Akin,Charles,16,16,Huckabee
5,Akin,Mike,16,16,Huckabee
6,Akin,Rebecca,16,16,Huckabee
7,Aldridge,Brittni,16,16,Huckabee
8,Allen,John D.,16,16,Huckabee
9,Allen,John D.,16,16,Huckabee


#### Somewhat organized example

In [54]:
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted].groupby('last_name_y').describe()

candidate_id                                             id        \
                   count  mean  std   min   25%   50%   75%   max count  mean   
last_name_y                                                                     
Clinton             25.0  34.0  0.0  34.0  34.0  34.0  34.0  34.0  25.0  34.0   
Giuliani            25.0  22.0  0.0  22.0  22.0  22.0  22.0  22.0  25.0  22.0   
Huckabee            25.0  16.0  0.0  16.0  16.0  16.0  16.0  16.0  25.0  16.0   
McCain              25.0  37.0  0.0  37.0  37.0  37.0  37.0  37.0  25.0  37.0   
Obama               25.0  20.0  0.0  20.0  20.0  20.0  20.0  20.0  25.0  20.0   
Paul                25.0  32.0  0.0  32.0  32.0  32.0  32.0  32.0  25.0  32.0   
Romney              25.0  35.0  0.0  35.0  35.0  35.0  35.0  35.0  25.0  35.0   

                                                
             std   min   25%   50%   75%   max  
last_name_y                                     
Clinton      0.0  34.0  34.0  34.0  34.0  34.0  
Giuliani     0.0  22.0  22.0  22.0  22.0  22.0  
Huckabee     0.0  16.0  16.0  16.0  16.0  16.0  
McCain       0.0  37.0  37.0  37.0  37.0  37.0  
Obama        0.0  20.0  20.0  20.0  20.0  20.0  
Paul         0.0  32.0  32.0  32.0  32.0  32.0  
Romney       0.0  35.0  35.0  35.0  35.0  35.0

### Other Joins with `pandas`
We didn't cover all possible joins because `SQLite` can only handle the few that we did discuss.  As mentioned, there are workarounds for some things in `SQLite`, but not evertyhing.  Fortunately, `pandas` can handle pretty much everything.  Here are a few joins that `pandas` can handle:
* `LEFT OUTER` (already discussed)
* `RIGHT OUTER` - Think of the "opposite" of a `LEFT OUTER` join (shade the intersection and *right* set in the Venn diagram).
* `FULL OUTER` - Combine everything from both tables (shade the entire Venn diagram)

#### Left Outer Join with `pandas`

In [55]:
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id", how="left")[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16,16,Huckabee
1,Ahrens,Don,16,16,Huckabee
2,Ahrens,Don,16,16,Huckabee
3,Ahrens,Don,16,16,Huckabee
4,Akin,Charles,16,16,Huckabee
5,Akin,Mike,16,16,Huckabee
6,Akin,Rebecca,16,16,Huckabee
7,Aldridge,Brittni,16,16,Huckabee
8,Allen,John D.,16,16,Huckabee
9,Allen,John D.,16,16,Huckabee


#### Right Outer Join with `pandas`

In [56]:
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id", how="right")[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16.0,16,Huckabee
1,Ahrens,Don,16.0,16,Huckabee
2,Ahrens,Don,16.0,16,Huckabee
3,Ahrens,Don,16.0,16,Huckabee
4,Akin,Charles,16.0,16,Huckabee
5,Akin,Mike,16.0,16,Huckabee
6,Akin,Rebecca,16.0,16,Huckabee
7,Aldridge,Brittni,16.0,16,Huckabee
8,Allen,John D.,16.0,16,Huckabee
9,Allen,John D.,16.0,16,Huckabee


#### Full Outer Join with `pandas`

In [57]:
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id", how="outer")[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16.0,16,Huckabee
1,Ahrens,Don,16.0,16,Huckabee
2,Ahrens,Don,16.0,16,Huckabee
3,Ahrens,Don,16.0,16,Huckabee
4,Akin,Charles,16.0,16,Huckabee
5,Akin,Mike,16.0,16,Huckabee
6,Akin,Rebecca,16.0,16,Huckabee
7,Aldridge,Brittni,16.0,16,Huckabee
8,Allen,John D.,16.0,16,Huckabee
9,Allen,John D.,16.0,16,Huckabee


# Exercises Part

## Exercises
1. Using an `INNER JOIN`, join the candidates and contributors tables by comparing the `candidate_id` and `candidates_id` columns.  Display your joined table with the columns `contributors.last_name`, `contributors.first_name`, and `candidates.last_name`.
2. Do the same inner join as in the last part, but this time append a `WHERE` clause to select a specific candidate's last name.

#### 1.

In [4]:
query = '''SELECT contributors.last_name, contributors.first_name, candidates.last_name 
           FROM contributors INNER JOIN candidates ON contributors.candidate_id = candidates.id'''
viz_tables(['contributors.last_name', 'contributors.first_name', 'candidates.last_name'], query)

,candidates.last_name,contributors.first_name,contributors.last_name
0,Huckabee,Steven,Agee
1,Huckabee,Don,Ahrens
2,Huckabee,Don,Ahrens
3,Huckabee,Don,Ahrens
4,Huckabee,Charles,Akin
5,Huckabee,Mike,Akin
6,Huckabee,Rebecca,Akin
7,Huckabee,Brittni,Aldridge
8,Huckabee,John D.,Allen
9,Huckabee,John D.,Allen


#### 2.

In [5]:
query = '''SELECT contributors.last_name, contributors.first_name, candidates.last_name 
           FROM contributors INNER JOIN candidates ON contributors.candidate_id = candidates.id WHERE candidates.last_name = "McCain"'''
viz_tables(['contributors.last_name', 'contributors.first_name', 'candidates.last_name'], query)

,candidates.last_name,contributors.first_name,contributors.last_name
0,McCain,ZAINUL,ABEDIN
1,McCain,SYBIL,ABBOTT
2,McCain,SYBIL,ABBOTT
3,McCain,RONALD,ABBOTT
4,McCain,RONALD,ABBOTT
5,McCain,ROBERT,ABBOTT
6,McCain,MIKE,ABBOTT
7,McCain,DAVID,ABBOT
8,McCain,PAULINE,ABBO
9,McCain,MARIA,ABATE


## Exercises
Use the following two tables to do the first two exercises in this section. Table A has the form:

| nA   | attr  | idA |
| :::: | ::::: | ::: |
| s1   | 23    | 0   |
| s2   | 7     | 2   |
| s3   | 15    | 2   |
| s4   | 31    | 0   |

and table B has the form:

| nB   | attr  | idB |
| :::: | ::::: | ::: |
| t1   | 60    | 0   |
| t2   | 14    | 7   |
| t3   | 22    | 2   |

1. Draw the table that would result from a `LEFT JOIN` using table A as the reference and the `id` columns for comparison.
2. Draw the table that would result from a `LEFT JOIN` using table B as the reference and the `id` columns for comparison.
3. Create a new table with the following form:

| average contribution | number of contributors | candidate last name |
| :::::::::::::::::::: | :::::::::::::::::::::: | ::::::::::::::::::: |
|    ...               |   ...                  |    ...              |

   The table should be created using the `LEFT JOIN` clause on the contributors table by joining the candidates table by the `id` column.  The `average contribution` column and `number of contributors` column should be obtained using the `AVG` and `COUNT` `SQL` functions.  Finally, you should use the `GROUP BY` clause on the candidates last name.

#### 1.

In [9]:
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS A")
cursor.execute("DROP TABLE IF EXISTS B")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE A (
               nA TEXT, 
               attr INTEGER,
               idA INTEGER NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE B (
               idB INTEGER NOT NULL, 
               nB TEXT, 
               attr INTEGER)''')

db.commit() # Commit changes to the database

cursor.execute('''INSERT INTO A 
          (nA, attr, idA)
          VALUES (?, ?, ?)''', ('s1', 23, 0))
cursor.execute('''INSERT INTO A 
          (nA, attr, idA)
          VALUES (?, ?, ?)''', ('s2', 7, 2))
cursor.execute('''INSERT INTO A 
          (nA, attr, idA)
          VALUES (?, ?, ?)''', ('s3', 15, 2))
cursor.execute('''INSERT INTO A 
          (nA, attr, idA)
          VALUES (?, ?, ?)''', ('s4', 31, 0))

cursor.execute('''INSERT INTO B 
          (nB, attr, idB)
          VALUES (?, ?, ?)''', ('t1', 60, 0))
cursor.execute('''INSERT INTO B 
          (nB, attr, idB)
          VALUES (?, ?, ?)''', ('t2', 14, 7))
cursor.execute('''INSERT INTO B 
          (nB, attr, idB)
          VALUES (?, ?, ?)''', ('t3', 22, 2))
db.commit() # Commit changes to the database

In [10]:
query = '''SELECT nA, A.attr, nB, B.attr FROM A LEFT JOIN B ON B.idB = A.idA'''
viz_tables(['nA', 'A.attr', 'nB', 'B.attr'], query)

,A.attr,B.attr,nA,nB
0,23,60,s1,t1
1,7,22,s2,t3
2,15,22,s3,t3
3,31,60,s4,t1


#### 2.

In [11]:
query = '''SELECT nA, A.attr, nB, B.attr FROM B LEFT JOIN A ON B.idB = A.idA'''
viz_tables(['nA', 'A.attr', 'nB', 'B.attr'], query)

,A.attr,B.attr,nA,nB
0,23.0,60,s1,t1
1,31.0,60,s4,t1
2,NaN,14,None,t2
3,7.0,22,s2,t3
4,15.0,22,s3,t3


#### 3.

In [12]:
query = '''SELECT AVG(contributors.amount), COUNT(contributors.id), candidates.last_name
           FROM contributors LEFT JOIN candidates ON contributors.candidate_id = candidates.id 
           GROUP BY candidates.last_name'''
viz_tables(['average contribution', 'number of contributors', 'candidate last name'], query)

,average contribution,candidate last name,number of contributors
0,50.6000,Clinton,25
1,-1360.0000,Giuliani,25
2,772.0000,Huckabee,25
3,452.6000,McCain,25
4,472.4356,Obama,25
5,-519.7088,Paul,25
6,156.0000,Romney,25


## Exercises
1. Use `pandas` to populate the contributors table.
2. Query the contributors tables with the following:
  1. List entries where the state is "VA" and the amount is less than $\$400.00$.
  2. List entries where the state is "NULL".
  3. List entries for the states of Texas and Pennsylvania.
  4. List entries where the amount contributed is between $\$10.00$ and $\$50.00$.

#### 1. Use pandas to populate the contributors table.

In [16]:
dfcontr.to_sql("contributors", dbp, if_exists="append", index=False)
query = '''SELECT * FROM contributors'''
csr.execute(query).fetchall()

[(1,
  'Agee',
  'Steven',
  None,
  '549 Laurel Branch Road',
  None,
  'Floyd',
  'VA',
  '24091',
  500.0,
  '2007-06-30',
  16),
 (2,
  'Ahrens',
  'Don',
  None,
  '4034 Rennellwood Way',
  None,
  'Pleasanton',
  'CA',
  '94566',
  250.0,
  '2007-05-16',
  16),
 (3,
  'Ahrens',
  'Don',
  None,
  '4034 Rennellwood Way',
  None,
  'Pleasanton',
  'CA',
  '94566',
  50.0,
  '2007-06-18',
  16),
 (4,
  'Ahrens',
  'Don',
  None,
  '4034 Rennellwood Way',
  None,
  'Pleasanton',
  'CA',
  '94566',
  100.0,
  '2007-06-21',
  16),
 (5,
  'Akin',
  'Charles',
  None,
  '10187 Sugar Creek Road',
  None,
  'Bentonville',
  'AR',
  '72712',
  100.0,
  '2007-06-16',
  16),
 (6,
  'Akin',
  'Mike',
  None,
  '181 Baywood Lane',
  None,
  'Monticello',
  'AR',
  '71655',
  1500.0,
  '2007-05-18',
  16),
 (7,
  'Akin',
  'Rebecca',
  None,
  '181 Baywood Lane',
  None,
  'Monticello',
  'AR',
  '71655',
  500.0,
  '2007-05-18',
  16),
 (8,
  'Aldridge',
  'Brittni',
  None,
  '808 Capitol Squa

#### 2. Query the contributors tables with the following:

##### A. List entries where the state is "VA" and the amount is less than  $400.00$ .

In [17]:
dfcontr.query("state=='VA' & amount < 400.0")

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


##### B. List entries where the state is "NULL"

In [20]:
dfcontr[dfcontr.state.isnull()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500.0,2008-11-20,35


##### C.List entries for the states of Texas and Pennsylvania.

In [22]:
dfcontr[dfcontr.state.isin(['TX', 'PA'])]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
65,BURGERT,RONALD,L.,5723 PLUMTREE DRIVE,NaN,DALLAS,TX,752524926,-1000.00,2008-03-05,22
70,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
71,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
79,Muse,Nina,Jo,2915 Toro Canyon Rd,NaN,Austin,TX,78746,-50.00,2008-04-21,32
93,Raught,Philip,M,4714 Plum Way,NaN,Pittsburgh,PA,15201,-1046.00,2008-04-21,32
94,Ferrara,Judith,D,1508 Waterford Road,NaN,Yardley,PA,19067,-1100.00,2008-04-21,32
95,Johnson,Cathleen,E.,1003 Justin Ln Apt 2016,NaN,Austin,TX,787572648,-14.76,2008-04-21,32
132,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-09-30,35
133,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,150.00,2007-08-09,35
134,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-07-19,35


##### D. List entries where the amount contributed is between  $10.00$  and  $50.00$ .

In [24]:
dfcontr.query("amount > 10.0 & amount < 50.0")

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
49,Harrison,Ryan,NaN,2247 3rd St,NaN,La Verne,CA,917504918,25.0,2007-07-26,20
101,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
140,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-09-17,35
143,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-08-06,35
144,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-07-10,35
158,ABBO,PAULINE,MORENCY,10720 JACOB LANE,NaN,WHITE LAKE,MI,483862274,35.0,2008-01-07,37
160,ABAIR,PETER,NaN,40 EVANS STREET,NaN,WATERTOWN,MA,24722150,25.0,2008-01-09,37


## Exercises
1. Sort the contributors table by `amount` and order in *descending* order.
2. Select the `first_name` and `amount` columns.
3. Select the `last_name` and `first_name` columns and drop duplicates.
4. Count how many there are after the duplicates have been dropped.

#### 1. Sort the contributors table by amount and order in descending order.

In [31]:
dfcontr.sort_values(by='amount', ascending=False)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.00,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.00,2008-01-25,37
15,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300.00,2007-06-12,16
33,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300.00,2007-09-30,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.00,2007-08-14,20
21,Baker,David,NaN,2550 Adamsbrooke Drive,NaN,Conway,AR,72034,2300.00,2007-04-11,16
13,Altes,R.D.,NaN,8600 Moody Road,NaN,Fort Smith,AR,72903,2300.00,2007-06-21,16
135,ABRAMOWITZ,NIRA,NaN,411 HARBOR ROAD,NaN,SOUTHPORT,CT,68901376,2300.00,2007-09-14,35
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.00,2007-05-18,16
174,ABRAHAM,SALEM,A.,P.O. BOX 7,NaN,CANADIAN,TX,790140007,1300.00,2008-01-30,37


#### 2. Select the first_name and amount columns.

In [32]:
dfcontr[['first_name', 'amount']]

,first_name,amount
0,Steven,500.0
1,Don,250.0
2,Don,50.0
3,Don,100.0
4,Charles,100.0
5,Mike,1500.0
6,Rebecca,500.0
7,Brittni,250.0
8,John D.,1000.0
9,John D.,1300.0


#### 3. Select the last_name and first_name columns and drop duplicates.

In [33]:
dfcontr[['last_name', 'first_name']].drop_duplicates()

,last_name,first_name
0,Agee,Steven
1,Ahrens,Don
4,Akin,Charles
5,Akin,Mike
6,Akin,Rebecca
7,Aldridge,Brittni
8,Allen,John D.
10,Allison,John W.
11,Allison,Rebecca
13,Altes,R.D.


#### 4. Count how many there are after the duplicates have been dropped.

In [35]:
dfcontr[['last_name', 'first_name']].drop_duplicates().count()

last_name     126
first_name    126
dtype: int64

## Exercises
1. Create a name column for the contributors table with field entries of the form "last name, first name"
2. For contributors from the state of "PA", change the name to "X".
3. Delete the newly created name column.

#### 1. Create a name column for the contributors table with field entries of the form "last name, first name"

In [41]:
dfcontr['name'] = dfcontr['last_name'] + ", " + dfcontr['first_name']
dfcontr

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16,"Akin, Mike"
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16,"Akin, Rebecca"
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16,"Aldridge, Brittni"
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16,"Allen, John D."
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16,"Allen, John D."


#### 2. For contributors from the state of "PA", change the name to "X".

In [43]:
dfcontr.loc[dfcontr.state == "PA", "name"] = "X"
dfcontr

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16,"Akin, Mike"
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16,"Akin, Rebecca"
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16,"Aldridge, Brittni"
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16,"Allen, John D."
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16,"Allen, John D."


#### 3.Delete the newly created name column.

In [44]:
del dfcontr['name']
dfcontr

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16


## Exercise
Use the `describe()` method on the `contributors` table.

In [46]:
dfcontr.describe()

,zip,amount,candidate_id
count,1.750000e+02,175.000000,175.000000
mean,3.780014e+08,3.418114,28.000000
std,3.628278e+08,1028.418999,7.823484
min,2.474000e+03,-2592.000000,16.000000
25%,9.336700e+04,-175.000000,20.000000
50%,3.233313e+08,100.000000,32.000000
75%,7.816946e+08,300.000000,35.000000
max,9.951532e+08,4600.000000,37.000000
